In [1]:
from langsmith import Client 

import os 
import openai 

from qdrant_client import QdrantClient 
from langchain_openai import ChatOpenAI  
from langchain_openai import OpenAIEmbeddings 

from ragas.llms import LangchainLLMWrapper 
from ragas.embeddings import LangchainEmbeddingsWrapper 

/home/k/AI-Engineering-Bootcamp/01-ai-engineering-bootcamp/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Download an example reference data point from LangSmith

In [2]:
client = Client()

In [3]:
dataset = client.read_dataset(dataset_name="rag-evaluation-dataset")

In [4]:
dataset

Dataset(name='rag-evaluation-dataset', description='Dataset for evaluating RAG pipeline', data_type=<DataType.kv: 'kv'>, id=UUID('7314766e-ca06-4404-8814-934b83990229'), created_at=datetime.datetime(2025, 10, 10, 2, 34, 29, 737011, tzinfo=datetime.timezone.utc), modified_at=datetime.datetime(2025, 10, 10, 2, 34, 29, 737011, tzinfo=datetime.timezone.utc), example_count=37, session_count=0, last_session_start_time=None, inputs_schema=None, outputs_schema=None, transformations=None, metadata={'runtime': {'sdk': 'langsmith-py', 'library': 'langsmith', 'runtime': 'python', 'platform': 'Linux-6.6.87.2-microsoft-standard-WSL2-x86_64-with-glibc2.31', 'sdk_version': '0.4.33', 'runtime_version': '3.12.11', 'langchain_version': None, 'py_implementation': 'CPython', 'langchain_core_version': '0.3.78'}})

In [5]:
list(client.list_examples(dataset_id=dataset.id, limit=10))[0].inputs

{'question': 'Do you sell power banks or portable battery chargers?'}

In [6]:
list(client.list_examples(dataset_id=dataset.id, limit=10))[0].outputs

{'ground_truth': 'We do not currently have any power banks or portable battery chargers listed in stock.',
 'reference_context_ids': [],
 'reference_description': []}

In [7]:
list(client.list_examples(dataset_id=dataset.id, limit=15))[11].inputs

{'question': 'Which products would be suitable if I want noise-canceling wireless earbuds?'}

In [8]:
list(client.list_examples(dataset_id=dataset.id, limit=15))[11].outputs

{'ground_truth': 'Options include TELSOR Wireless Earbuds (B0C6K1GQCF) and S23-vine Wireless Earbuds (B0B9FTVL58), both featuring noise-canceling tech and long battery life. RUSAM GA33 Game Earbuds (B09WCFC5D9) also offer noise cancellation and are suitable for gaming and music.',
 'reference_context_ids': ['B0C6K1GQCF', 'B0B9FTVL58', 'B09WCFC5D9'],
 'reference_description': ["TELSOR Wireless Earbuds for iPhone, Bluetooth Headphones Touch Control Stereo Sound Bluetooth Earbuds with Noise Cancelling Mic for Calls, 30H Playtime, IPX7 Waterproof Earbuds for Android, Black ♬【Bluetooth】Pair instantly with an uninterrupted and stable transmission with Bluetooth 5.1. AVRCP, HCP, HSP, and A2DP profiles are supported. The wireless earbuds are compatible with most Bluetooth enabled iPhones, Andriods, smart TVs, computers, etc. Each wireless earbuds will pair with each other when they are removed from the charging case. From here, enable Bluetooth on your chosen device and pair with the headphone

In [9]:
reference_input = list(client.list_examples(dataset_id=dataset.id, limit=15))[11].inputs
reference_output = list(client.list_examples(dataset_id=dataset.id, limit=15))[11].outputs 

### Rag Pipeline

In [10]:
qdrant_client = QdrantClient(url="http://localhost:6333") 

In [11]:
def get_embedding(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input = text,
        model = model,
    )

    return response.data[0].embedding

def retrieve_data(query, qdrant_client, k=5): 
    query_embedding = get_embedding(query)  
    results = qdrant_client.query_points(
        collection_name="Amazon-items-collection-00",  
        query=query_embedding, 
        limit=k,  
    ) 
    retrieved_context_ids = []
    retrieved_context = [] 
    similarity_scores = [] 
    
    for result in results.points: 
        retrieved_context_ids.append(result.payload["parent_asin"]) 
        retrieved_context.append(result.payload["description"]) 
        similarity_scores.append(result.score) 

    return {
        "retrieved_context_ids": retrieved_context_ids, 
        "retrieved_context": retrieved_context, 
        "similarity_scores": similarity_scores, 
    }

def process_context(context):
    formatted_context = "" 

    for id, chunk in zip(context["retrieved_context_ids"], context["retrieved_context"]):  
        formatted_context += f"- {id}: {chunk}\n"  

    return formatted_context

def build_prompt(preprocessed_context, question): 

    prompt = f""" 
    You are a shopping assistant that can answer qeustions about the products in stock. 

    You will be given a question and a list of context. 

    Instructions: 
    - You need to answer the question based on the provided context only. 
    - Never use word context and refer to it as the available products. 

    Context: 
    {preprocessed_context} 

    Question: 
    {question} 
    """ 
    return prompt 

def generate_answer(prompt):  
    response = openai.chat.completions.create(
        model="gpt-4o-mini", 
        messages=[{"role": "system", "content": prompt}], 
        temperature=0.5, 
    ) 

    return response.choices[0].message.content

def rag_pipeline(question, qdrant_client,top_k=5): 

    retrieved_context = retrieve_data(question, qdrant_client, top_k) 
    preprocessed_context = process_context(retrieved_context)
    prompt = build_prompt(preprocessed_context, question)
    answer = generate_answer(prompt)

    final_result = {
        "answer": answer, 
        "question": question,  
        "retrieved_context_ids": retrieved_context["retrieved_context_ids"],   
        "retrieved_context": retrieved_context["retrieved_context"],   
        "similarity_scores": retrieved_context["similarity_scores"],   
    }
    return final_result

In [12]:
rag_pipeline("Can I get some charger?", qdrant_client, top_k=5)

{'answer': 'Yes, there are several charger options available:\n\n1. **iPhone Charger Cord Lightning Cables**: A pack of three 3-foot long Apple MFi Certified USB A charging cables suitable for various iPhone models, including iPhone 13, 12, 11, and more. They are designed for durability and fast charging.\n\n2. **5 in 1 USB C to Multi Charging Cable**: A 10-foot multi charging cable that includes connectors for Lightning, Type C, and Micro USB, allowing you to charge multiple devices simultaneously.\n\n3. **MUXA iPhone Charger**: A 6-pack of colorful nylon Lightning cables in various lengths (3, 6, and 10 FT) that are MFi certified and compatible with many iPhone models.\n\n4. **USB C to Lightning Cable**: A 6-foot long MFi certified cable designed for fast charging and data transfer, compatible with various iPhone models.\n\n5. **Charger Cable for Fitbit Inspire 3**: A 2-pack of 3.3-foot charging cables specifically designed for the Fitbit Inspire 3.\n\nLet me know if you need more in

### RAGAS Metrics

In [13]:
from ragas.dataset_schema import SingleTurnSample 
from ragas.metrics import IDBasedContextPrecision, IDBasedContextRecall, Faithfulness, ResponseRelevancy  
# from ragas.metrics import Faithfulness, ResponseRelevancy  

ragas_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-mini"))  
ragas_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model="text-embedding-3-small")) 

/tmp/ipykernel_18443/2032601054.py:5: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use the modern LLM providers instead: from ragas.llms.base import llm_factory; llm = llm_factory('gpt-4o-mini') or from ragas.llms.base import instructor_llm_factory; llm = instructor_llm_factory('openai', client=openai_client)
  ragas_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-mini"))
/tmp/ipykernel_18443/2032601054.py:6: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  ragas_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model="text-embedding-3-small"))


In [14]:
reference_input 

{'question': 'Which products would be suitable if I want noise-canceling wireless earbuds?'}

In [22]:
reference_output

{'ground_truth': 'Options include TELSOR Wireless Earbuds (B0C6K1GQCF) and S23-vine Wireless Earbuds (B0B9FTVL58), both featuring noise-canceling tech and long battery life. RUSAM GA33 Game Earbuds (B09WCFC5D9) also offer noise cancellation and are suitable for gaming and music.',
 'reference_context_ids': ['B0C6K1GQCF', 'B0B9FTVL58', 'B09WCFC5D9'],
 'reference_description': ["TELSOR Wireless Earbuds for iPhone, Bluetooth Headphones Touch Control Stereo Sound Bluetooth Earbuds with Noise Cancelling Mic for Calls, 30H Playtime, IPX7 Waterproof Earbuds for Android, Black ♬【Bluetooth】Pair instantly with an uninterrupted and stable transmission with Bluetooth 5.1. AVRCP, HCP, HSP, and A2DP profiles are supported. The wireless earbuds are compatible with most Bluetooth enabled iPhones, Andriods, smart TVs, computers, etc. Each wireless earbuds will pair with each other when they are removed from the charging case. From here, enable Bluetooth on your chosen device and pair with the headphone

In [15]:
result = rag_pipeline(reference_input["question"], qdrant_client)

In [16]:
result

{'answer': 'The suitable products for noise-canceling wireless earbuds are:\n\n1. TELSOR Wireless Earbuds (B0C6K1GQCF) - These earbuds feature a noise-cancelling mic for calls and provide clear sound quality, making them a great choice for noise reduction.\n\n2. RUSAM GA33 A Enhanced Edition TWS Game Earbuds (B09WCFC5D9) - These earbuds offer HD physical noise cancellation, which helps in reducing ambient noise for an enhanced audio experience.',
 'question': 'Which products would be suitable if I want noise-canceling wireless earbuds?',
 'retrieved_context_ids': ['B0C6K1GQCF',
  'B0CBMPG524',
  'B0B9FTVL58',
  'B09WCFC5D9',
  'B0B14HTZ59'],
 'retrieved_context': ["TELSOR Wireless Earbuds for iPhone, Bluetooth Headphones Touch Control Stereo Sound Bluetooth Earbuds with Noise Cancelling Mic for Calls, 30H Playtime, IPX7 Waterproof Earbuds for Android, Black ♬【Bluetooth】Pair instantly with an uninterrupted and stable transmission with Bluetooth 5.1. AVRCP, HCP, HSP, and A2DP profiles ar

In [17]:
async def ragas_faithfulness(run, example):

    sample = SingleTurnSample(
            user_input=run["question"],
            response=run["answer"],
            retrieved_contexts=run["retrieved_context"]
        )
    scorer = Faithfulness(llm=ragas_llm)

    return await scorer.single_turn_ascore(sample)

In [18]:
await ragas_faithfulness(result, "")

0.5

In [19]:
async def ragas_responce_relevancy(run, example):

    sample = SingleTurnSample(
            user_input=run["question"],
            response=run["answer"],
            retrieved_contexts=run["retrieved_context"]
        )
    scorer = ResponseRelevancy(llm=ragas_llm, embeddings=ragas_embeddings)

    return await scorer.single_turn_ascore(sample)

In [20]:
await ragas_responce_relevancy(result, "")

np.float64(0.9491993509247144)

In [21]:
async def ragas_context_precision_id_based(run, example):

    sample = SingleTurnSample(
            retrieved_context_ids=run["retrieved_context_ids"],
            reference_context_ids=example["reference_context_ids"]
        )
    scorer = IDBasedContextPrecision()

    return await scorer.single_turn_ascore(sample)

In [23]:
await ragas_context_precision_id_based(result, reference_output)

0.6

In [24]:
async def ragas_context_recall_id_based(run, example):

    sample = SingleTurnSample(
            retrieved_context_ids=run["retrieved_context_ids"],
            reference_context_ids=example["reference_context_ids"]
        )
    scorer = IDBasedContextRecall()

    return await scorer.single_turn_ascore(sample)

In [25]:
await ragas_context_recall_id_based(result, reference_output)

1.0